In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

EMBED_ROOT = Path("/vol/biomedic3/data/EMBED")

In [2]:
# Need instance number for our specific  casewise aggregation methodologies
full_dicom = pd.read_csv(
    EMBED_ROOT / "tables/EMBED_OpenData_metadata.csv", low_memory=False
)[
    [
        "InstanceNumber",
        "anon_dicom_path",
        "PixelSpacing",
        "ImagerPixelSpacing",
        "Rows",
        "Columns",
    ]
]

In [3]:
dicom = pd.read_csv(
    EMBED_ROOT / "tables/EMBED_OpenData_metadata_reduced.csv", low_memory=False
)
print(len(dicom))
dicom = dicom.merge(full_dicom, on="anon_dicom_path")
print(len(dicom))
dicom["image_path"] = (
    dicom["empi_anon"].astype("str")
    + "/"
    + dicom["anon_dicom_path"].str.split("/").str[-1].str.split(".dcm").str[0]
    + ".png"
)

480323
480323


In [4]:
# XCCL shouldn't be converted to CC so manually editing it
dicom.loc[
    (dicom["SeriesDescription"] == "RXCCL") | (dicom["SeriesDescription"] == "LXCCL"),
    "ViewPosition",
] = "XCCL"

# Getting all rows with "ViewPosition" == Nan (but for which SeriesDescription is also not nan, as these are the ones subject to the data entry error)
view_nan = dicom.loc[
    (dicom.ViewPosition.isna()) & (dicom.SeriesDescription.isna() == False)
]

# Drop these rows from
dicom_no_nans = dicom[~dicom.index.isin(view_nan.index)]

view_nan["ViewPosition"] = view_nan["SeriesDescription"].apply(
    lambda x: "CC" if "CC" in x else ("MLO" if "MLO" in x else None)
)

dicom = pd.concat([dicom_no_nans, view_nan], axis=0, ignore_index=True)

/tmp/ipykernel_34030/2960478528.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  view_nan['ViewPosition'] = view_nan['SeriesDescription'].apply(lambda x: 'CC' if 'CC' in x else ('MLO' if 'MLO' in x else None))


In [5]:
print(len(dicom))
# Remove any duplicated images
dicom = dicom.drop_duplicates(subset="anon_dicom_path")
# Remove spot compressed and magnified images
dicom = dicom[dicom.spot_mag.isna()]
# Remove invalid views
dicom = dicom[dicom.ViewPosition.isin(["CC", "MLO"])]
# Remove images from male clients
dicom = dicom[dicom.PatientSex == "F"]
print(len(dicom))

480323
420457


In [6]:
# Remove any unnecessary fields from the DICOM imagewise dataframe (this may need to be updated in the future if other fields are deemed relevant)
dicom = dicom[
    [
        "empi_anon",
        "acc_anon",
        "image_path",
        "FinalImageType",
        "ImageLateralityFinal",
        "ViewPosition",
        "Manufacturer",
        "ManufacturerModelName",
    ]
]

In [7]:
# Conversion dictionary to standardised naming of various fields in clincial metadata

# Human reader BIRADS density assessment
dens_conversion = {1.0: "A", 2.0: "B", 3.0: "C", 4.0: "D"}

In [8]:
# Load in the clinical metadata
mag = pd.read_csv(EMBED_ROOT / "tables/EMBED_OpenData_clinical.csv", low_memory=False)
print(len(mag))
# Remove cases from cases a valid BIRADS density assessment
mag = mag[mag.tissueden.isin([1.0, 2.0, 3.0, 4.0])]
mag.replace({"tissueden": dens_conversion}, inplace=True)

81776


In [9]:
# Keep important study metadata tags to join up with final aggregated dataframe at end of script
mag = mag[["empi_anon", "tissueden", "study_date_anon", "acc_anon"]].drop_duplicates(
    subset="acc_anon"
)
print(len(mag))

72188


In [10]:
# Convert to pandas datetime object
mag["study_date_anon"] = pd.to_datetime(mag["study_date_anon"], errors="coerce")

In [11]:
dicom.Manufacturer.value_counts()

Manufacturer
HOLOGIC, Inc.           386257
GE MEDICAL SYSTEMS       25569
FUJIFILM Corporation      8133
GE HEALTHCARE              498
Name: count, dtype: int64

In [12]:
# Only consider studies which have a valid link between the DICOM and clinical metadata
print(len(dicom))
df = mag.merge(dicom, on="acc_anon")
print(len(df))

420457
418819


In [13]:
df.columns

Index(['empi_anon_x', 'tissueden', 'study_date_anon', 'acc_anon',
       'empi_anon_y', 'image_path', 'FinalImageType', 'ImageLateralityFinal',
       'ViewPosition', 'Manufacturer', 'ManufacturerModelName'],
      dtype='object')

In [14]:
# df.to_csv('joined_simple.csv', index=False)